In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import glob
import os
import warnings

from astropy.io import ascii
from astropy.io import fits

%matplotlib inline

In [2]:
igrins_log = pd.read_csv('./reduced_log.csv') # RRSIA reduced data log
igrins_log = igrins_log.drop(labels=['SNRH','SNRK','ROTPA','AM','BVC'],axis=1) # drop a few columns

luhman_cat = pd.read_table('./apjs328021t7_sptype_ascii.txt',header=0)
luhman_sptypes = luhman_cat['SED_Class']

class_3_mask = luhman_sptypes == 'III'
luhman_class3 = luhman_cat[class_3_mask]
luhman_class3['Name']

0       HBC 358 A+B+C
2             HBC 359
3             HBC 360
4             HBC 361
6             HBC 362
            ...      
340           HBC 427
341              ... 
346           CIDA 10
348              ... 
349    RXJ 05072+2437
Name: Name, Length: 123, dtype: object

In [3]:
luhman_class3

,2MASS,Name,Spectral_Type,alpha(2-8 mum),alpha(2-24 mum),alpha(3.6-8 mum),alpha(3.6-24 mum),SED_Class,Unnamed: 8
0,J04034930+2610520,HBC 358 A+B+C,M3.5,-2.49/-2.49,-2.68/-2.68,-2.74/-2.74,-2.83/-2.83,III,NaN
2,J04035084+2610531,HBC 359,M2,-2.58/-2.58,-2.77/-2.77,-2.74/-2.74,-2.88/-2.88,III,NaN
3,J04043936+2158186,HBC 360,M3.5,-2.54/-2.55,...,-2.75/-2.75,...,III,NaN
4,J04043984+2158215,HBC 361,M3,-2.55/-2.55,...,-2.73/-2.73,...,III,NaN
6,J04053087+2151106,HBC 362,M2,-2.58/-2.58,-2.63/-2.63,-2.77/-2.77,-2.73/-2.73,III,NaN
...,...,...,...,...,...,...,...,...,...
340,J04560201+3021037,HBC 427,K5,-2.72/-2.75,-2.69/-2.71,-2.77/-2.79,-2.71/-2.72,III,NaN
341,J04574903+3015195,...,M9.25,-2.29/-2.29,...,-2.76/-2.76,...,III,NaN
346,J05061674+2446102,CIDA 10,M4,-2.50/-2.54,...,-2.66/-2.67,...,III,NaN
348,J05064662+2104296,...,M5.25,-2.40/-2.42,...,-2.67/-2.68,...,III,NaN


In [4]:
# unique targets of observation from RRISA
unique_rrisa = pd.Series(igrins_log['OBJNAME_super'].unique())
unique_rrisa

0            LHS 3325
1                vega
2           NLTT45791
3           NLTT46858
4              GJ797B
            ...      
5264           AQ PUP
5265        HIP 46633
5266           AQ-CAR
5267           YZ-CAR
5268    TIC 168426455
Length: 5269, dtype: object

In [5]:
match_mask = luhman_class3['Name'].isin(unique_rrisa) # check the names from RRISA against Luhman class 3 objects
luhman_class3 = luhman_class3[match_mask].sort_values(['Spectral_Type']) # just some sorting

class3_objects = luhman_class3['Name'] # just the names of class 3 objects that are also in RRISA (allegedly)

In [14]:
luhman_class3

,2MASS,Name,Spectral_Type,alpha(2-8 mum),alpha(2-24 mum),alpha(3.6-8 mum),alpha(3.6-24 mum),SED_Class,Unnamed: 8
314,J04455129+1555496,HD 30171,G5,-2.72/-2.75,...,-2.80/-2.81,...,III,NaN
96,J04215884+2818066,HD 283572,G5,-2.71/-2.74,-2.83/-2.85,-2.77/-2.79,-2.89/-2.89,III,NaN
327,J04553695+3017553,LkCa 19,K0,-2.72/-2.78,-2.46/-2.49,-2.77/-2.79,-2.41/-2.42,III,NaN
340,J04560201+3021037,HBC 427,K5,-2.72/-2.75,-2.69/-2.71,-2.77/-2.79,-2.71/-2.72,III,NaN
349,J05071206+2437163,RXJ 05072+2437,K6,-2.68/-2.71,...,-2.76/-2.77,...,III,NaN
38,J04162810+2807358,LkCa 4,K7,-2.57/-2.59,-2.74/-2.75,-2.60/-2.61,-2.79/-2.80,III,NaN
60,J04184703+2820073,Hubble 4,K7,-2.53/-2.64,-2.65/-2.71,-2.61/-2.67,-2.71/-2.74,III,NaN
209,J04331003+2433433,V830 Tau,K7,-2.71/-2.74,-2.81/-2.83,-2.79/-2.80,-2.87/-2.88,III,NaN
199,J04324373+1802563,L1551-55,K7,-2.64/-2.67,-2.80/-2.81,-2.74/-2.76,-2.88/-2.89,III,NaN
182,J04321456+1820147,V827 Tau,K7,-2.64/-2.67,-2.75/-2.76,-2.74/-2.75,-2.81/-2.82,III,NaN


In [36]:
import requests
import ast

igrins_class3 = igrins_log[igrins_log['OBJNAME_super'].isin(luhman_class3['Name'])]
igrins_class3['URLS_K']

430     {'20141125/SDCK_20141125_0127.sn.fits': 'https...
461     {'20141127/SDCK_20141127_0104.sn.fits': 'https...
463     {'20141127/SDCK_20141127_0112.sn.fits': 'https...
518     {'20141129/SDCK_20141129_0129.sn.fits': 'https...
530     {'20141130/SDCK_20141130_0145.sn.fits': 'https...
                              ...                        
8921    {'20190309/SDCK_20190309_0035.sn.fits': 'https...
8924    {'20190309/SDCK_20190309_0041.sn.fits': 'https...
8934    {'20190314/SDCK_20190314_0039.sn.fits': 'https...
8943    {'20190316/SDCK_20190316_0033.sn.fits': 'https...
8947    {'20190316/SDCK_20190316_0041.sn.fits': 'https...
Name: URLS_K, Length: 280, dtype: object